In [ ]:
import os
!git clone https://github.com/TensorSpeech/TensorFlowTTS.git
os.chdir("TensorFlowTTS")
!pip install .
os.chdir("..")
import sys
sys.path.append("TensorFlowTTS/")

In [ ]:
import tensorflow as tf

import yaml
import numpy as np
import matplotlib.pyplot as plt

import IPython.display as ipd

from tensorflow_tts.inference import AutoConfig
from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoProcessor

In [ ]:
import os
os.chdir('/content/drive/My Drive/Project/TTS_YIN/TensorFlowTTS')
os.getcwd()
os.listdir()

In [ ]:
## 파일이름 변경 => 충돌방지 

# from pathlib import Path
# import shutil

# for path in Path(os.getcwd()).rglob('*.py'):
#   flis = list()
#   with open(path,"r") as f:
#     flis = f.readlines()
  
#   rep = open(path,"w")
#   for fl in flis:
#     nuli = fl
#     if "examples." in fl:
#       nuli = nuli.replace("examples.","ttsexamples.")
#       print(nuli.strip())


#     rep.write(nuli)
  
#   rep.close()

# shutil.move("examples","ttsexamples")

In [ ]:
# 데이터 전처리 
!tensorflow-tts-preprocess --rootdir ./kss --outdir ./dump_kss --config ./preprocess/kss_preprocess.yaml --dataset kss

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2020-10-14 10:09:43,457 (preprocess:196) INFO: 3_0365 sampling rate is 44100, not 24000, we resample it.
2020-10-14 10:09:43,709 (preprocess:196) INFO: 3_1605 sampling rate is 44100, not 24000, we resample it.
2020-10-14 10:09:43,889 (preprocess:196) INFO: 3_3259 sampling rate is 44100, not 24000, we resample it.
[Preprocessing train]:  65% 7890/12210 [1:48:54<53:01,  1.36it/s]2020-10-14 10:09:49,537 (preprocess:196) INFO: 4_4652 sampling rate is 44100, not 24000, we resample it.
2020-10-14 10:09:49,832 (preprocess:196) INFO: 3_4663 sampling rate is 44100, not 24000, we resample it.
2020-10-14 10:09:50,106 (preprocess:196) INFO: 4_1536 sampling rate is 44100, not 24000, we resample it.
[Preprocessing train]:  65% 7891/12210 [1:48:54<54:04,  1.33it/s]2020-10-14 10:09:50,462 (preprocess:196) INFO: 4_3928 sampling rate is 44100, not 24000, we resample it.
2020-10-14 10:09:50,773 (preprocess:196) INFO: 4_5120 sampling rate is 44100, not 24000, we resampl

In [ ]:
# 데이터 Normailizing 
!tensorflow-tts-normalize --rootdir ./dump_kss --outdir ./dump_kss --config ./preprocess/kss_preprocess.yaml --dataset kss

2020-10-14 23:35:06.000145: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
2020-10-14 23:35:30,385 (preprocess:521) INFO: Files to normalize: 12853
[Normalizing]: 100% 12853/12853 [1:06:05<00:00,  3.24it/s]


In [ ]:
# 타코트론 학습
!CUDA_VISIBLE_DEVICES=0 python ./ttsexamples/tacotron2/train_tacotron2.py\
  --train-dir ./dump_kss/train/ \
  --dev-dir ./dump_kss/valid/ \
  --outdir ./ttsexamples/tacotron2/exp/train.tacotron2.v1/ \
  --config ./ttsexamples/tacotron2/conf/tacotron2.kss.v1.yaml \
  --use-norm 1 \
  --mixed_precision 0 \
  --resume ./ttsexamples/tacotron2/exp/train.tacotron2.v1/checkpoints/ckpt-28000

2020-09-29 08:46:06.336790: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-29 08:46:07.637403: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-09-29 08:46:07.685869: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-29 08:46:07.686449: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-09-29 08:46:07.686492: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-29 08:46:07.923354: I tensorflow/stream_executor/pl

In [ ]:
# melgan 학습 
!CUDA_VISIBLE_DEVICES=0 python ./ttsexamples/multiband_melgan/train_multiband_melgan.py\
  --train-dir ./dump_kss/train/ \
  --dev-dir ./dump_kss/valid/ \
  --outdir ./ttsexamples/multiband_melgan/exp/train.multiband_melgan.v1/ \
  --config ./ttsexamples/multiband_melgan/conf/multiband_melgan.v1.yaml \
  --use-norm 1 \
  --generator_mixed_precision 1 \
  --resume ''